# Lab4

### Zad1

In [252]:
from unidecode import unidecode
import spacy
from spacy.tokenizer import Tokenizer
import random

In [101]:
def sigma(a,b):
    return not a == b

def edit_length(x, y, f_sigma):
    m, n = len(x), len(y)
    edit = [[0]*(n+1) for i in range(m+1)]
    
    edit[0][0] = (0, "Nothing")
    for i in range(1,m+1):
        edit[i][0] = (i, 1)
    for j in range(1,n+1):
        edit[0][j] = (j, 2)
    
    if n == 0 and m == 0:
        return edit[0][0][0], edit[0][0][1]
    
    # 1 delete , 2 insert, 3 change, 4 pass
    for i in range(1,m+1):
        for j in range(1,n+1):
            a = edit[i-1][j][0] + 1
            b = edit[i][j-1][0] + 1
            change_value = f_sigma(x[i-1], y[j-1])
            c = edit[i-1][j-1][0] + change_value
            if a < b and a < c:
                edit[i][j] = (a, 1)
            elif b < c:
                edit[i][j] = (b, 2)
            else:
                if change_value:
                    edit[i][j] = (c, 3)
                else:
                    edit[i][j] = (c, 4)

    path = []
    i = m
    j = n
    while i != 0 or j != 0:
        path_value = edit[i][j][1]
        if path_value == 1:
            path.append("Delete")
            i -= 1
        elif path_value == 2:
            path.append("Insert")
            j -= 1
        elif path_value == 3:
            path.append("Change")
            i -= 1
            j -= 1
        else:
            path.append("Pass")
            i -= 1
            j -= 1
            
    path.reverse()
    return edit[m][n][0], path
    

### Zad2

In [7]:
def x2y_visualisation(x, y):
    length, path = edit_length(x, y, sigma)
    print(path)
    x_idx = 0
    y_idx = 0
    print("{} => {}".format(x, y))
    for action in path:
        if action == "Insert":
            print("{}*{}*{} insert {}".format(x[0:x_idx],y[y_idx], x[x_idx:], y[y_idx]))
            x = x[0:x_idx] + y[y_idx] + x[x_idx:]
        elif action == "Delete":
            print("{}*{}*{} delete {}".format(x[0:x_idx], x[x_idx], x[x_idx+1:], x[x_idx]))
            x = x[0:x_idx] + x[x_idx+1:]
            x_idx -= 1
            y_idx -= 1
        elif action == "Change":
            print("{}*{}*{} Change {} -> {}".format(x[0:x_idx], y[y_idx], x[x_idx+1:], x[x_idx], y[y_idx]))
            x = x[0:x_idx] + y[y_idx] + x[x_idx+1:]
        x_idx += 1
        y_idx += 1
    print(x)

### Zad3

In [161]:
x2y_visualisation("los", "kloc")

['Insert', 'Pass', 'Pass', 'Change']
los => kloc
*k*los insert k
klo*c* Change s -> c
kloc


In [9]:
x2y_visualisation("Łódź", "Lodz")

['Change', 'Change', 'Pass', 'Change']
Łódź => Lodz
*L*ódź Change Ł -> L
L*o*dź Change ó -> o
Lod*z* Change ź -> z
Lodz


In [10]:
x2y_visualisation("kwintesencja", "quintessence")

['Change', 'Change', 'Pass', 'Pass', 'Pass', 'Pass', 'Insert', 'Pass', 'Pass', 'Pass', 'Pass', 'Delete', 'Change']
kwintesencja => quintessence
*q*wintesencja Change k -> q
q*u*intesencja Change w -> u
quinte*s*sencja insert s
quintessenc*j*a delete j
quintessenc*e* Change a -> e
quintessence


In [11]:
x2y_visualisation("ATGAATCTTACCGCCTCG", "ATGAGGCTCTGGCCCCTG")

['Pass', 'Pass', 'Pass', 'Pass', 'Change', 'Change', 'Pass', 'Pass', 'Insert', 'Pass', 'Change', 'Change', 'Pass', 'Change', 'Pass', 'Pass', 'Pass', 'Delete', 'Pass']
ATGAATCTTACCGCCTCG => ATGAGGCTCTGGCCCCTG
ATGA*G*TCTTACCGCCTCG Change A -> G
ATGAG*G*CTTACCGCCTCG Change T -> G
ATGAGGCT*C*TACCGCCTCG insert C
ATGAGGCTCT*G*CCGCCTCG Change A -> G
ATGAGGCTCTG*G*CGCCTCG Change C -> G
ATGAGGCTCTGGC*C*CCTCG Change G -> C
ATGAGGCTCTGGCCCCT*C*G delete C
ATGAGGCTCTGGCCCCTG


### Zad4

### Computing subsequence, x_diff, y_diff

In [151]:
def lcs(x, y):
    m, n = len(x), len(y)
    edit = [[0]*(n+1) for i in range(m+1)]
    
    edit[0][0] = (0, "")
    for i in range(1,m+1):
        edit[i][0] = (0, 1)
    for j in range(1,n+1):
        edit[0][j] = (0, 2)
    
    if n == 0 and m == 0:
        return edit[0][0][0], edit[0][0][1]
    
    # 1 delete , 2 insert, 3 change,4 append lcs (diagonal)
    for i in range(1,m+1):
        for j in range(1,n+1):
            a = edit[i-1][j][0]
            b = edit[i][j-1][0]
            change_value = (x[i-1] == y[j-1])
            c = edit[i-1][j-1][0] + change_value
            if a > b and a > c:
                edit[i][j] = (a, 1)
            elif b > c:
                edit[i][j] = (b, 2)
            else:
                if change_value:
                    edit[i][j] = (c, 4)
                else:
                    edit[i][j] = (c, 3)
    lcs = []
    x_diff = []
    y_diff = []
    i = m
    j = n
    while i != 0 or j != 0:
        path_value = edit[i][j][1]
        if path_value == 1:
            x_diff.append(x[i-1])
            i -= 1
        elif path_value == 2:
            y_diff.append(y[j-1])
            j -= 1
        elif path_value == 3:
            x_diff.append(x[i-1])
            y_diff.append(y[j-1])
            i -= 1
            j -= 1
        elif path_value == 4:
            lcs.append(y[j-1])
            i -= 1
            j -= 1
            
    lcs.reverse()
    x_diff.reverse()
    y_diff.reverse()
    return edit[m][n][0], "".join(lcs), "".join(x_diff), "".join(y_diff)

In [354]:
lcs("rower", "oxwe")

(3, 'owe', 'rr', 'x')

### Computing length

In [14]:
def lcs_sigma(a, b):
    if a == b:
        return 0
    else:
        return 2

def lcs_length(x, y):
    return int((len(x) + len(y) - edit_length(x,y,lcs_sigma)[0]) / 2)
    

In [213]:
lcs_length("rower", "oxwe")

3

### Zad5

### Read file

In [352]:
with open("romeo-i-julia-700.txt", "r", encoding="utf8") as file:
    lines = file.readlines() # lines
    
nlp = spacy.load("pl_core_news_sm")

# Tokenize text line by line
tokens_1 = [[] for i in range(len(lines))]
tokens_2 = [[] for i in range(len(lines))]
tokens_number = 0
n = len(lines)
for i in range(n):
    line_tokens = list(nlp.tokenizer(lines[i]))
    tokens_1[i] = line_tokens.copy()
    tokens_2[i] = line_tokens.copy()
    tokens_number += len(line_tokens)

In [353]:
print("Number of tokens : ", tokens_number)

Number of tokens :  3053


### Remove 3% of tokens

In [349]:
tokens_number_after_remove = int(0.97 * tokens_number)
while tokens_number != tokens_number_after_remove:
    i = random.randrange(len(lines))
    j = random.randrange(len(lines))
    
    n = len(tokens_1[i])
    m = len(tokens_2[j])
    
    if n == 0 or m == 0:
        continue
    
    k = random.randrange(n)
    l = random.randrange(m)
    
    if tokens_1[i][k].text == '\n' or tokens_2[j][l].text == '\n':
        continue
    
    tokens_1[i].pop(k)
    tokens_2[j].pop(l)
    
    tokens_number -= 1

### Longest common subsequence of tokens

In [338]:
t_1 = []
t_2 = []
for i in range(len(lines)):
    t_1 += tokens_1[i]
    t_2 += tokens_2[i]

lcs_length(t_1, t_2)

2877

### Zad8

In [340]:
def print_diff_line(diff, index, file):
    if len(diff) > 0:
        if diff[len(diff) - 1] == '\n': # remove end of line in diff if n < m
            diff = diff[:-1]
        if len(diff) > 0:
            print("<line:{}, '{}'>:{}".format(index, file, diff))

def diff(file_1, file_2):
    with open(file_1, "r", encoding="utf8") as f:
        lines_1 = f.readlines()
        
    with open(file_2, "r", encoding="utf8") as f:
        lines_2 = f.readlines()
    
    n = len(lines_1)
    m = len(lines_2)
    
    
    for i in range(n):
        if i < m:
            lcs_length, lcs_subsequence, x_diff, y_diff = lcs(lines_1[i], lines_2[i])
            print_diff_line(x_diff, i, file_1)
            print_diff_line(y_diff, i, file_2)
        else:
            print_diff_line(lines_1[i], i, file_1) # print rest of the first file
        
    for i in range(n, m):
        print_diff_line(lines_2[i], i, file_2) # print rest of the second file

text_1.txt:
<br>rower
<br>abcdefgh
<br>jojojo

text_2.txt:
<br>oxwe
<br>bcxxdefgh
<br>ojyyjo
<br>ergAGE

In [341]:
diff("text_1.txt", "text_2.txt")

<line:0, 'text_1.txt'>:rr
<line:0, 'text_2.txt'>:x
<line:1, 'text_1.txt'>:a
<line:1, 'text_2.txt'>:xx
<line:2, 'text_1.txt'>:jo
<line:2, 'text_2.txt'>:yy
<line:3, 'text_2.txt'>:ergAGE


### Zad9

### Write tokenized text to files

In [350]:
with open("tokens_1.txt", "w+", encoding="utf8") as f:
    for line_tokens in tokens_1:
        f.write(" ".join(map(str,line_tokens)))
    
with open("tokens_2.txt", "w+", encoding="utf8") as f:
    for line_tokens in tokens_2:
        f.write(" ".join(map(str,line_tokens)))

In [351]:
diff("tokens_1.txt", "tokens_2.txt")

<line:2, 'tokens_2.txt'>:Romeo 
<line:5, 'tokens_1.txt'>:88 -
<line:5, 'tokens_2.txt'>: -903
<line:11, 'tokens_2.txt'>: szlachetnego
<line:13, 'tokens_1.txt'>:  
<line:13, 'tokens_2.txt'>: —
<line:17, 'tokens_2.txt'>: —
<line:18, 'tokens_1.txt'>: *
<line:20, 'tokens_1.txt'>:  
<line:21, 'tokens_1.txt'>:  
<line:25, 'tokens_1.txt'>:  
<line:28, 'tokens_1.txt'>: Montekiego
<line:29, 'tokens_2.txt'>: KAPULET
<line:51, 'tokens_1.txt'>: z
<line:53, 'tokens_2.txt'>: rodzicielskie
<line:57, 'tokens_2.txt'>: ,
<line:58, 'tokens_1.txt'>: przedstawienia
<line:61, 'tokens_2.txt'>: …
<line:82, 'tokens_1.txt'>: bobyśmy
<line:97, 'tokens_2.txt'>: zwyczaj
<line:117, 'tokens_1.txt'>:a kżdego
<line:127, 'tokens_2.txt'>: od
<line:132, 'tokens_2.txt'>:t ylko
<line:142, 'tokens_1.txt'>:z predsiębrać
<line:157, 'tokens_1.txt'>:SAMSON 
<line:159, 'tokens_1.txt'>: dobyty .
<line:162, 'tokens_1.txt'>:GRZEGORZ 
<line:179, 'tokens_1.txt'>: za
<line:184, 'tokens_2.txt'>: ;
<line:187, 'tokens_2.txt'>:SAMSON 
<lin